In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from pycaret.regression import *
import joblib

# Column 제거

In [5]:
train_df = pd.read_csv('../data/train.csv')
train_df.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,0,1,106,지방도1112호선,0,0,60.0,0.0,32400.0,0.0,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,0,2,103,일반국도11호선,0,0,60.0,0.0,0.0,0.0,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,0,2,103,일반국도16호선,0,0,80.0,0.0,0.0,0.0,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,0,2,107,태평로,0,0,50.0,0.0,0.0,0.0,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,0,2,103,일반국도12호선,0,0,80.0,0.0,0.0,0.0,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [6]:
train_df.columns

Index(['id', 'base_date', 'day_of_week', 'base_hour', 'road_in_use',
       'lane_count', 'road_rating', 'road_name', 'multi_linked',
       'connect_code', 'maximum_speed_limit', 'vehicle_restricted',
       'weight_restricted', 'height_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target'],
      dtype='object')

In [7]:
test_df = pd.read_csv('../data/test.csv')
test_df.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,목,17,0,3,107,연삼로,0,0,70.0,0.0,0.0,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음
1,TEST_000001,20220809,화,12,0,2,103,일반국도12호선,0,0,70.0,0.0,0.0,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음
2,TEST_000002,20220805,금,2,0,1,103,일반국도16호선,0,0,60.0,0.0,0.0,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음
3,TEST_000003,20220818,목,23,0,3,103,일반국도11호선,0,0,70.0,0.0,0.0,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음
4,TEST_000004,20220810,수,17,0,3,106,번영로,0,0,70.0,0.0,0.0,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음


In [8]:
# ## pycaret을 위해 줄이기
# train_df = train_df.sample(frac=1, ignore_index=True)
# test_df = test_df.sample(frac=1, ignore_index=True)

# train_df = train_df.iloc[:int(0.1*len(train_df))]
# test_df = test_df[:int(0.1*len(train_df))]

In [9]:
drop_col = ['id', 'vehicle_restricted', 'height_restricted', 'start_latitude', 'start_longitude', #'base_date'
            'end_latitude', 'end_longitude', 'road_in_use']
x = train_df.drop(drop_col, axis = 1)
x = x.drop('target', axis = 1)
y = train_df.loc[:, 'target']

test_data = test_df.drop(drop_col, axis = 1)

In [10]:
# import statsmodels.api as sm
# for i in train_df.columns:
#     model = sm.Logit(train_df[i], train_df['target'])
#     result = model.fit()
#     print(result.pvalues)

In [11]:
dumm_cols = ['day_of_week', 'base_hour', 'road_rating', 'road_name', 
             'multi_linked', 'connect_code', 'road_type', 
             'start_turn_restricted', 'end_turn_restricted', 'start_node_name', 'end_node_name']

In [12]:
# for col in dumm_cols:
#     ohe = OneHotEncoder(sparse=False) # 나중에 drop 적용해보기
    
#     temp_x = ohe.fit_transform(x[col].values.reshape(-1,1))
# #     temp_x_df = pd.DataFrame(temp_x, columns=ohe.categories_[0])
#     temp_x_df = pd.DataFrame(temp_x, columns=ohe.get_feature_names([col]))
#     x.drop(col, axis=1, inplace = True)
#     x = pd.concat([x, temp_x_df], axis = 1)
    
#     temp_test = ohe.transform(test_data[col].values.reshape(-1,1))
# #     temp_test_df = pd.DataFrame(temp_test, columns=ohe.categories_[0])
#     temp_test_df = pd.DataFrame(temp_test, columns=ohe.get_feature_names([col]))
#     test_data.drop(col, axis=1, inplace = True)
#     test_data = pd.concat([test_data, temp_test_df], axis = 1)

In [13]:
for col in dumm_cols:
    le = LabelEncoder()
    x[col]= le.fit_transform(x[col])
    test_data[col]= le.transform(test_data[col])

In [14]:
x.shape

(4701217, 15)

In [15]:
test_data.shape

(291241, 15)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 2022)

In [17]:
# x_train.to_csv('../data/train1_origin/x_train.csv', index=False)
# x_val.to_csv('../data/train1_origin/x_val.csv', index=False)
# y_train.to_csv('../data/train1_origin/y_train.csv', index=False)
# y_val.to_csv('../data/train1_origin/y_val.csv', index=False)
# test_data.to_csv('../data/train1_origin/x_test.csv', index=False)

# Catboost

In [64]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

In [81]:
params = {'depth'         : [4,5,6,7,8,9, 10],
          'learning_rate' : [0.01,0.02,0.03,0.04],
          'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]
         }
cat = CatBoostRegressor(loss_function='MAE', verbose=0)
grid_cat = GridSearchCV(estimator=cat, param_grid = params, cv = 3, n_jobs=-1, verbose=1)

grid_cat.fit(x_train, y_train)

Fitting 3 folds for each of 280 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed: 85.8min
[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed: 101.4min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f894ba2e1c0>,
             iid='deprecated', n_jobs=-1,
             param_grid={'depth': [4, 5, 6, 7, 8, 9, 10],
                         'iterations': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                        100],
                         'learning_rate': [0.01, 0.02, 0.03, 0.04]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [91]:
pred = grid_cat.predict(x_val)
pred
print(mean_absolute_error(y_val, pred))

6.0735307649754


In [82]:
grid_cat.best_params_

{'depth': 10, 'iterations': 100, 'learning_rate': 0.04}

In [83]:
y_pred = grid_cat.predict(test_data)
result = pd.DataFrame({'id':test_df['id'], 'target':y_pred})
result.head()

,id,target
0,TEST_000000,27.862911
1,TEST_000001,44.084466
2,TEST_000002,55.354193
3,TEST_000003,34.846257
4,TEST_000004,40.970338


In [84]:
result.to_csv('../data/submission_v6.csv', index=False)

In [95]:
model_cat = CatBoostRegressor()
model_cat.fit(x_train, y_train)

Learning rate set to 0.150341
0:	learn: 14.8144593	total: 104ms	remaining: 1m 44s
1:	learn: 13.9175486	total: 201ms	remaining: 1m 40s
2:	learn: 13.2211945	total: 296ms	remaining: 1m 38s
3:	learn: 12.6856126	total: 392ms	remaining: 1m 37s
4:	learn: 12.2529243	total: 488ms	remaining: 1m 37s
5:	learn: 11.8605600	total: 583ms	remaining: 1m 36s
6:	learn: 11.5675107	total: 678ms	remaining: 1m 36s
7:	learn: 11.3337530	total: 770ms	remaining: 1m 35s
8:	learn: 11.1417422	total: 867ms	remaining: 1m 35s
9:	learn: 10.9477010	total: 962ms	remaining: 1m 35s
10:	learn: 10.7983845	total: 1.05s	remaining: 1m 34s
11:	learn: 10.6195481	total: 1.16s	remaining: 1m 35s
12:	learn: 10.4834155	total: 1.25s	remaining: 1m 35s
13:	learn: 10.3686107	total: 1.35s	remaining: 1m 34s
14:	learn: 10.2670049	total: 1.45s	remaining: 1m 34s
15:	learn: 10.1939180	total: 1.53s	remaining: 1m 34s
16:	learn: 10.0745507	total: 1.64s	remaining: 1m 34s
17:	learn: 9.9988338	total: 1.74s	remaining: 1m 35s
18:	learn: 9.9095501	total:

159:	learn: 7.0820653	total: 15.1s	remaining: 1m 19s
160:	learn: 7.0729650	total: 15.2s	remaining: 1m 19s
161:	learn: 7.0628169	total: 15.3s	remaining: 1m 19s
162:	learn: 7.0544159	total: 15.4s	remaining: 1m 19s
163:	learn: 7.0434339	total: 15.5s	remaining: 1m 19s
164:	learn: 7.0375031	total: 15.6s	remaining: 1m 18s
165:	learn: 7.0280085	total: 15.7s	remaining: 1m 18s
166:	learn: 7.0176930	total: 15.8s	remaining: 1m 18s
167:	learn: 7.0070133	total: 15.9s	remaining: 1m 18s
168:	learn: 7.0037924	total: 16s	remaining: 1m 18s
169:	learn: 6.9989000	total: 16.1s	remaining: 1m 18s
170:	learn: 6.9939794	total: 16.2s	remaining: 1m 18s
171:	learn: 6.9868948	total: 16.2s	remaining: 1m 18s
172:	learn: 6.9847205	total: 16.3s	remaining: 1m 18s
173:	learn: 6.9796733	total: 16.4s	remaining: 1m 17s
174:	learn: 6.9736631	total: 16.5s	remaining: 1m 17s
175:	learn: 6.9641241	total: 16.6s	remaining: 1m 17s
176:	learn: 6.9570279	total: 16.7s	remaining: 1m 17s
177:	learn: 6.9513278	total: 16.8s	remaining: 1m

316:	learn: 6.2996798	total: 30.2s	remaining: 1m 5s
317:	learn: 6.2961358	total: 30.3s	remaining: 1m 4s
318:	learn: 6.2921917	total: 30.4s	remaining: 1m 4s
319:	learn: 6.2895414	total: 30.5s	remaining: 1m 4s
320:	learn: 6.2845834	total: 30.6s	remaining: 1m 4s
321:	learn: 6.2824849	total: 30.7s	remaining: 1m 4s
322:	learn: 6.2796945	total: 30.8s	remaining: 1m 4s
323:	learn: 6.2759025	total: 30.9s	remaining: 1m 4s
324:	learn: 6.2736390	total: 31s	remaining: 1m 4s
325:	learn: 6.2692088	total: 31.1s	remaining: 1m 4s
326:	learn: 6.2658146	total: 31.2s	remaining: 1m 4s
327:	learn: 6.2645059	total: 31.3s	remaining: 1m 4s
328:	learn: 6.2631522	total: 31.4s	remaining: 1m 3s
329:	learn: 6.2612925	total: 31.5s	remaining: 1m 3s
330:	learn: 6.2580786	total: 31.6s	remaining: 1m 3s
331:	learn: 6.2538906	total: 31.7s	remaining: 1m 3s
332:	learn: 6.2499805	total: 31.7s	remaining: 1m 3s
333:	learn: 6.2482485	total: 31.8s	remaining: 1m 3s
334:	learn: 6.2460871	total: 31.9s	remaining: 1m 3s
335:	learn: 6.

478:	learn: 5.9314444	total: 45.8s	remaining: 49.8s
479:	learn: 5.9299614	total: 45.9s	remaining: 49.7s
480:	learn: 5.9282067	total: 46s	remaining: 49.6s
481:	learn: 5.9267664	total: 46.1s	remaining: 49.5s
482:	learn: 5.9253060	total: 46.2s	remaining: 49.4s
483:	learn: 5.9235972	total: 46.3s	remaining: 49.3s
484:	learn: 5.9208715	total: 46.4s	remaining: 49.2s
485:	learn: 5.9196715	total: 46.5s	remaining: 49.1s
486:	learn: 5.9181884	total: 46.6s	remaining: 49.1s
487:	learn: 5.9166609	total: 46.7s	remaining: 49s
488:	learn: 5.9153339	total: 46.8s	remaining: 48.9s
489:	learn: 5.9133833	total: 46.8s	remaining: 48.8s
490:	learn: 5.9123630	total: 46.9s	remaining: 48.7s
491:	learn: 5.9099489	total: 47s	remaining: 48.6s
492:	learn: 5.9084879	total: 47.1s	remaining: 48.5s
493:	learn: 5.9056060	total: 47.3s	remaining: 48.4s
494:	learn: 5.9042872	total: 47.4s	remaining: 48.3s
495:	learn: 5.9024837	total: 47.4s	remaining: 48.2s
496:	learn: 5.8995092	total: 47.5s	remaining: 48.1s
497:	learn: 5.8983

639:	learn: 5.7178551	total: 1m 1s	remaining: 34.6s
640:	learn: 5.7175924	total: 1m 1s	remaining: 34.5s
641:	learn: 5.7169058	total: 1m 1s	remaining: 34.4s
642:	learn: 5.7158672	total: 1m 1s	remaining: 34.3s
643:	learn: 5.7146604	total: 1m 1s	remaining: 34.2s
644:	learn: 5.7137685	total: 1m 1s	remaining: 34.1s
645:	learn: 5.7127989	total: 1m 2s	remaining: 34s
646:	learn: 5.7117852	total: 1m 2s	remaining: 33.9s
647:	learn: 5.7104172	total: 1m 2s	remaining: 33.8s
648:	learn: 5.7096295	total: 1m 2s	remaining: 33.7s
649:	learn: 5.7089485	total: 1m 2s	remaining: 33.6s
650:	learn: 5.7075666	total: 1m 2s	remaining: 33.5s
651:	learn: 5.7067919	total: 1m 2s	remaining: 33.4s
652:	learn: 5.7055601	total: 1m 2s	remaining: 33.3s
653:	learn: 5.7036648	total: 1m 2s	remaining: 33.3s
654:	learn: 5.7024413	total: 1m 2s	remaining: 33.2s
655:	learn: 5.7013430	total: 1m 3s	remaining: 33.1s
656:	learn: 5.7002622	total: 1m 3s	remaining: 33s
657:	learn: 5.6993364	total: 1m 3s	remaining: 32.9s
658:	learn: 5.69

798:	learn: 5.5794771	total: 1m 17s	remaining: 19.4s
799:	learn: 5.5789253	total: 1m 17s	remaining: 19.3s
800:	learn: 5.5786937	total: 1m 17s	remaining: 19.2s
801:	learn: 5.5780741	total: 1m 17s	remaining: 19.1s
802:	learn: 5.5775038	total: 1m 17s	remaining: 19s
803:	learn: 5.5768151	total: 1m 17s	remaining: 18.9s
804:	learn: 5.5761781	total: 1m 17s	remaining: 18.9s
805:	learn: 5.5754928	total: 1m 17s	remaining: 18.8s
806:	learn: 5.5749428	total: 1m 18s	remaining: 18.7s
807:	learn: 5.5743635	total: 1m 18s	remaining: 18.6s
808:	learn: 5.5740058	total: 1m 18s	remaining: 18.5s
809:	learn: 5.5734332	total: 1m 18s	remaining: 18.4s
810:	learn: 5.5728176	total: 1m 18s	remaining: 18.3s
811:	learn: 5.5720822	total: 1m 18s	remaining: 18.2s
812:	learn: 5.5717731	total: 1m 18s	remaining: 18.1s
813:	learn: 5.5708952	total: 1m 18s	remaining: 18s
814:	learn: 5.5701490	total: 1m 18s	remaining: 17.9s
815:	learn: 5.5696417	total: 1m 18s	remaining: 17.8s
816:	learn: 5.5692702	total: 1m 18s	remaining: 17.

954:	learn: 5.4841307	total: 1m 32s	remaining: 4.36s
955:	learn: 5.4833798	total: 1m 32s	remaining: 4.26s
956:	learn: 5.4828846	total: 1m 32s	remaining: 4.17s
957:	learn: 5.4825904	total: 1m 32s	remaining: 4.07s
958:	learn: 5.4818770	total: 1m 32s	remaining: 3.97s
959:	learn: 5.4813630	total: 1m 33s	remaining: 3.88s
960:	learn: 5.4807357	total: 1m 33s	remaining: 3.78s
961:	learn: 5.4802780	total: 1m 33s	remaining: 3.68s
962:	learn: 5.4790434	total: 1m 33s	remaining: 3.58s
963:	learn: 5.4787048	total: 1m 33s	remaining: 3.49s
964:	learn: 5.4784549	total: 1m 33s	remaining: 3.39s
965:	learn: 5.4780251	total: 1m 33s	remaining: 3.29s
966:	learn: 5.4776185	total: 1m 33s	remaining: 3.2s
967:	learn: 5.4770826	total: 1m 33s	remaining: 3.1s
968:	learn: 5.4765082	total: 1m 33s	remaining: 3s
969:	learn: 5.4755790	total: 1m 34s	remaining: 2.91s
970:	learn: 5.4750646	total: 1m 34s	remaining: 2.81s
971:	learn: 5.4745957	total: 1m 34s	remaining: 2.71s
972:	learn: 5.4736986	total: 1m 34s	remaining: 2.62

In [96]:
pred = model_cat.predict(x_val)
pred
print(mean_absolute_error(y_val, pred))

3.984917124333543


In [116]:
joblib.dump(model_cat, 'models/model_cat.pkl')

['models/model_cat.pkl']

# RF

In [101]:
model_rf = RandomForestRegressor()

model_rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [102]:
pred = model_rf.predict(x_val)
pred
print(mean_absolute_error(y_val, pred))

3.589763242950743


In [114]:
joblib.dump(model_rf, 'models/model_rf.pkl')

['models/model_rf.pkl']

In [121]:
y_pred = model_rf.predict(test_data)
result = pd.DataFrame({'id':test_df['id'], 'target':y_pred})
result.head()

,id,target
0,TEST_000000,25.660000
1,TEST_000001,41.020000
2,TEST_000002,67.080000
3,TEST_000003,38.710000
4,TEST_000004,43.136667


In [124]:
result.to_csv('../data/submission_v7.csv', index=False)

# XGB

In [97]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(n_estimators=100,gamma=1,eta=0.1,max_depth=5,reg_lambda=5,reg_alpha=5,random_state=2022)

model_xgb.fit(x_train, y_train)



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=1, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=2022, ...)

In [98]:
from xgboost import XGBRegressor

model_xgb_v2 = XGBRegressor()

model_xgb_v2.fit(x_train, y_train)



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, ...)

In [99]:
pred = model_xgb.predict(x_val)
pred
print(mean_absolute_error(y_val, pred))

5.875256047947337


In [100]:
pred = model_xgb_v2.predict(x_val)
pred
print(mean_absolute_error(y_val, pred))

4.14413145262452


In [115]:
joblib.dump(model_xgb_v2, 'models/model_xgb_v2.pkl')

['models/model_xgb_v2.pkl']

In [16]:
y_pred = model_xgb.predict(test_data)

In [18]:
result = pd.DataFrame({'id':test_df['id'], 'target':y_pred})
result.head()

,id,target
0,TEST_000000,28.699699
1,TEST_000001,45.003902
2,TEST_000002,55.764816
3,TEST_000003,31.750568
4,TEST_000004,43.394955


# Stacking

In [20]:
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

rf = RandomForestRegressor()
xgb  = XGBRegressor()
cat = CatBoostRegressor()
rf_meta = RandomForestRegressor()

estimators = [('rf', rf), ('xgb', xgb), ('catboost', cat)]

model = StackingRegressor(estimators=estimators,
               final_estimator=rf_meta,
               verbose=1,)

model.fit(x_train, y_train).score(x_train, y_train) # score: R2 score 반환

Learning rate set to 0.150341
0:	learn: 14.8144593	total: 154ms	remaining: 2m 33s
1:	learn: 13.9175486	total: 251ms	remaining: 2m 5s
2:	learn: 13.2211945	total: 346ms	remaining: 1m 54s
3:	learn: 12.6856126	total: 441ms	remaining: 1m 49s
4:	learn: 12.2529243	total: 536ms	remaining: 1m 46s
5:	learn: 11.8605600	total: 630ms	remaining: 1m 44s
6:	learn: 11.5675107	total: 725ms	remaining: 1m 42s
7:	learn: 11.3337530	total: 815ms	remaining: 1m 41s
8:	learn: 11.1417422	total: 912ms	remaining: 1m 40s
9:	learn: 10.9477010	total: 1.01s	remaining: 1m 39s
10:	learn: 10.7983845	total: 1.1s	remaining: 1m 38s
11:	learn: 10.6195481	total: 1.2s	remaining: 1m 38s
12:	learn: 10.4834155	total: 1.3s	remaining: 1m 38s
13:	learn: 10.3686107	total: 1.39s	remaining: 1m 37s
14:	learn: 10.2670049	total: 1.49s	remaining: 1m 38s
15:	learn: 10.1939180	total: 1.58s	remaining: 1m 37s
16:	learn: 10.0745507	total: 1.69s	remaining: 1m 37s
17:	learn: 9.9988338	total: 1.79s	remaining: 1m 37s
18:	learn: 9.9095501	total: 1.8

159:	learn: 7.0820653	total: 15s	remaining: 1m 18s
160:	learn: 7.0729650	total: 15.1s	remaining: 1m 18s
161:	learn: 7.0628169	total: 15.2s	remaining: 1m 18s
162:	learn: 7.0544159	total: 15.4s	remaining: 1m 18s
163:	learn: 7.0434339	total: 15.5s	remaining: 1m 18s
164:	learn: 7.0375031	total: 15.5s	remaining: 1m 18s
165:	learn: 7.0280085	total: 15.6s	remaining: 1m 18s
166:	learn: 7.0176930	total: 15.7s	remaining: 1m 18s
167:	learn: 7.0070133	total: 15.8s	remaining: 1m 18s
168:	learn: 7.0037924	total: 15.9s	remaining: 1m 18s
169:	learn: 6.9989000	total: 16s	remaining: 1m 18s
170:	learn: 6.9939794	total: 16.1s	remaining: 1m 18s
171:	learn: 6.9868948	total: 16.2s	remaining: 1m 17s
172:	learn: 6.9847205	total: 16.3s	remaining: 1m 17s
173:	learn: 6.9796733	total: 16.4s	remaining: 1m 17s
174:	learn: 6.9736631	total: 16.5s	remaining: 1m 17s
175:	learn: 6.9641241	total: 16.6s	remaining: 1m 17s
176:	learn: 6.9570279	total: 16.7s	remaining: 1m 17s
177:	learn: 6.9513278	total: 16.8s	remaining: 1m 1

316:	learn: 6.2996798	total: 30.2s	remaining: 1m 4s
317:	learn: 6.2961358	total: 30.3s	remaining: 1m 4s
318:	learn: 6.2921917	total: 30.4s	remaining: 1m 4s
319:	learn: 6.2895414	total: 30.4s	remaining: 1m 4s
320:	learn: 6.2845834	total: 30.5s	remaining: 1m 4s
321:	learn: 6.2824849	total: 30.6s	remaining: 1m 4s
322:	learn: 6.2796945	total: 30.7s	remaining: 1m 4s
323:	learn: 6.2759025	total: 30.8s	remaining: 1m 4s
324:	learn: 6.2736390	total: 30.9s	remaining: 1m 4s
325:	learn: 6.2692088	total: 31s	remaining: 1m 4s
326:	learn: 6.2658146	total: 31.1s	remaining: 1m 4s
327:	learn: 6.2645059	total: 31.2s	remaining: 1m 3s
328:	learn: 6.2631522	total: 31.3s	remaining: 1m 3s
329:	learn: 6.2612925	total: 31.4s	remaining: 1m 3s
330:	learn: 6.2580786	total: 31.5s	remaining: 1m 3s
331:	learn: 6.2538906	total: 31.6s	remaining: 1m 3s
332:	learn: 6.2499805	total: 31.7s	remaining: 1m 3s
333:	learn: 6.2482485	total: 31.8s	remaining: 1m 3s
334:	learn: 6.2460871	total: 31.9s	remaining: 1m 3s
335:	learn: 6.

478:	learn: 5.9314444	total: 45.7s	remaining: 49.7s
479:	learn: 5.9299614	total: 45.8s	remaining: 49.6s
480:	learn: 5.9282067	total: 45.9s	remaining: 49.5s
481:	learn: 5.9267664	total: 46s	remaining: 49.5s
482:	learn: 5.9253060	total: 46.1s	remaining: 49.4s
483:	learn: 5.9235972	total: 46.2s	remaining: 49.3s
484:	learn: 5.9208715	total: 46.3s	remaining: 49.2s
485:	learn: 5.9196715	total: 46.4s	remaining: 49.1s
486:	learn: 5.9181884	total: 46.5s	remaining: 49s
487:	learn: 5.9166609	total: 46.6s	remaining: 48.9s
488:	learn: 5.9153339	total: 46.7s	remaining: 48.8s
489:	learn: 5.9133833	total: 46.8s	remaining: 48.7s
490:	learn: 5.9123630	total: 46.9s	remaining: 48.6s
491:	learn: 5.9099489	total: 47s	remaining: 48.5s
492:	learn: 5.9084879	total: 47.1s	remaining: 48.4s
493:	learn: 5.9056060	total: 47.2s	remaining: 48.3s
494:	learn: 5.9042872	total: 47.3s	remaining: 48.2s
495:	learn: 5.9024837	total: 47.4s	remaining: 48.1s
496:	learn: 5.8995092	total: 47.5s	remaining: 48s
497:	learn: 5.898321

638:	learn: 5.7184719	total: 1m 1s	remaining: 34.5s
639:	learn: 5.7178551	total: 1m 1s	remaining: 34.4s
640:	learn: 5.7175924	total: 1m 1s	remaining: 34.3s
641:	learn: 5.7169058	total: 1m 1s	remaining: 34.2s
642:	learn: 5.7158672	total: 1m 1s	remaining: 34.1s
643:	learn: 5.7146604	total: 1m 1s	remaining: 34s
644:	learn: 5.7137685	total: 1m 1s	remaining: 33.9s
645:	learn: 5.7127989	total: 1m 1s	remaining: 33.8s
646:	learn: 5.7117852	total: 1m 1s	remaining: 33.7s
647:	learn: 5.7104172	total: 1m 1s	remaining: 33.7s
648:	learn: 5.7096295	total: 1m 2s	remaining: 33.6s
649:	learn: 5.7089485	total: 1m 2s	remaining: 33.5s
650:	learn: 5.7075666	total: 1m 2s	remaining: 33.4s
651:	learn: 5.7067919	total: 1m 2s	remaining: 33.3s
652:	learn: 5.7055601	total: 1m 2s	remaining: 33.2s
653:	learn: 5.7036648	total: 1m 2s	remaining: 33.1s
654:	learn: 5.7024413	total: 1m 2s	remaining: 33s
655:	learn: 5.7013430	total: 1m 2s	remaining: 32.9s
656:	learn: 5.7002622	total: 1m 2s	remaining: 32.8s
657:	learn: 5.69

...

631:	learn: 5.7165925	total: 48.2s	remaining: 28.1s
632:	learn: 5.7157146	total: 48.3s	remaining: 28s
633:	learn: 5.7144942	total: 48.3s	remaining: 27.9s
634:	learn: 5.7140939	total: 48.4s	remaining: 27.8s
635:	learn: 5.7132657	total: 48.5s	remaining: 27.8s
636:	learn: 5.7125465	total: 48.6s	remaining: 27.7s
637:	learn: 5.7113288	total: 48.7s	remaining: 27.6s
638:	learn: 5.7107269	total: 48.7s	remaining: 27.5s
639:	learn: 5.7100988	total: 48.8s	remaining: 27.5s
640:	learn: 5.7089918	total: 48.9s	remaining: 27.4s
641:	learn: 5.7072828	total: 48.9s	remaining: 27.3s
642:	learn: 5.7066080	total: 49s	remaining: 27.2s
643:	learn: 5.7056331	total: 49.1s	remaining: 27.1s
644:	learn: 5.7045790	total: 49.2s	remaining: 27.1s
645:	learn: 5.7036721	total: 49.3s	remaining: 27s
646:	learn: 5.7030827	total: 49.3s	remaining: 26.9s
647:	learn: 5.7020659	total: 49.4s	remaining: 26.8s
648:	learn: 5.7014109	total: 49.5s	remaining: 26.8s
649:	learn: 5.6996359	total: 49.6s	remaining: 26.7s
650:	learn: 5.6988

790:	learn: 5.5782635	total: 1m	remaining: 15.9s
791:	learn: 5.5775095	total: 1m	remaining: 15.8s
792:	learn: 5.5767723	total: 1m	remaining: 15.8s
793:	learn: 5.5762599	total: 1m	remaining: 15.7s
794:	learn: 5.5752895	total: 1m	remaining: 15.6s
795:	learn: 5.5745676	total: 1m	remaining: 15.5s
796:	learn: 5.5738174	total: 1m	remaining: 15.5s
797:	learn: 5.5733300	total: 1m	remaining: 15.4s
798:	learn: 5.5726325	total: 1m	remaining: 15.3s
799:	learn: 5.5714604	total: 1m	remaining: 15.2s
800:	learn: 5.5706951	total: 1m 1s	remaining: 15.2s
801:	learn: 5.5699084	total: 1m 1s	remaining: 15.1s
802:	learn: 5.5692021	total: 1m 1s	remaining: 15s
803:	learn: 5.5683856	total: 1m 1s	remaining: 14.9s
804:	learn: 5.5680446	total: 1m 1s	remaining: 14.9s
805:	learn: 5.5673093	total: 1m 1s	remaining: 14.8s
806:	learn: 5.5660448	total: 1m 1s	remaining: 14.7s
807:	learn: 5.5656461	total: 1m 1s	remaining: 14.6s
808:	learn: 5.5647887	total: 1m 1s	remaining: 14.5s
809:	learn: 5.5636109	total: 1m 1s	remaining

949:	learn: 5.4762449	total: 1m 12s	remaining: 3.81s
950:	learn: 5.4759001	total: 1m 12s	remaining: 3.73s
951:	learn: 5.4755314	total: 1m 12s	remaining: 3.66s
952:	learn: 5.4752958	total: 1m 12s	remaining: 3.58s
953:	learn: 5.4746300	total: 1m 12s	remaining: 3.51s
954:	learn: 5.4739016	total: 1m 12s	remaining: 3.43s
955:	learn: 5.4734826	total: 1m 12s	remaining: 3.35s
956:	learn: 5.4731643	total: 1m 12s	remaining: 3.28s
957:	learn: 5.4728287	total: 1m 13s	remaining: 3.2s
958:	learn: 5.4722605	total: 1m 13s	remaining: 3.13s
959:	learn: 5.4719971	total: 1m 13s	remaining: 3.05s
960:	learn: 5.4710930	total: 1m 13s	remaining: 2.97s
961:	learn: 5.4707880	total: 1m 13s	remaining: 2.9s
962:	learn: 5.4703338	total: 1m 13s	remaining: 2.82s
963:	learn: 5.4700140	total: 1m 13s	remaining: 2.74s
964:	learn: 5.4697720	total: 1m 13s	remaining: 2.67s
965:	learn: 5.4693515	total: 1m 13s	remaining: 2.59s
966:	learn: 5.4689557	total: 1m 13s	remaining: 2.52s
967:	learn: 5.4684021	total: 1m 13s	remaining: 2

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  6.4min finished


0.939675704151639

In [21]:
pred = model.predict(x_val)
pred
print(mean_absolute_error(y_val, pred))

3.686703022883399


In [23]:
y_pred = model.predict(test_data)
result = pd.DataFrame({'id':test_df['id'], 'target':y_pred})
result.head()

,id,target
0,TEST_000000,24.680000
1,TEST_000001,43.930000
2,TEST_000002,63.922500
3,TEST_000003,36.970000
4,TEST_000004,42.354333


In [24]:
result.to_csv('../data/submission_v8.csv', index=False)

# Jae Kyeong's

In [21]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRFRegressor
from lightgbm import LGBMRegressor
# from lightgbm import DaskLGBMClassifier

In [24]:
rf_model = RandomForestRegressor(n_estimators=300,
              max_depth=30,
              max_leaf_nodes=32,
              random_state=42,
              n_jobs=-1
              )

xgb_model = XGBRFRegressor(max_depth=30,
                n_estimators=300,
                learning_rate=0.01,
                gamma=0, subsample=0.75,
                use_label_encoder=False,
                colsample_bytree=1,
                eval_metric='logloss',
                n_jobs=-1
                # subsample=0.7
                )

xgb_meta = XGBRFRegressor(max_depth=27,
                n_estimators=500,
                learning_rate=0.005,
                gamma=0, subsample=0.75,
                use_label_encoder=False,
                colsample_bytree=1,
                eval_metric='logloss',
                n_jobs=-1
                # subsample=0.7
                )

lgb_model = LGBMRegressor(
                max_depth= 30,
                n_estimators= 300,
                learning_rate= 0.001,
                num_leaves = 40,
                n_jobs=-1)

# Second Layer
lgb_meta = LGBMRegressor(
                max_depth= 25,
                n_estimators= 600,
                learning_rate= 0.001,
                num_leaves = 32,
                n_jobs=-1
                )

mlp_model = MLPRegressor(hidden_layer_sizes=(32,), random_state=1, max_iter=10, validation_fraction=0.15, warm_start=False)



In [25]:
vr = VotingRegressor([('lgb', lgb_meta), ('mlp', mlp_model)], n_jobs=-1)

In [ ]:
estimators = [('rf', rf_model), ('xgb', xgb_model), ('lgb_1', lgb_model)]

model = StackingRegressor(estimators=estimators,
               final_estimator=vr,
               verbose=1,
               n_jobs=-1)

model.fit(x_train, y_train).score(x_train, y_train) # score: R2 score 반환

# Pycaret

In [38]:
jeju_model = setup(session_id = 2022, data = pd.concat([x_train,y_train],axis=1), target = 'target',
                   test_data = pd.concat([x_val,y_val],axis=1),
                   normalize = True, normalize_method = 'zscore',
                   transformation=True, 
                   #fold_strategy='stratifiedkfold', 
                   use_gpu = True)

,Description,Value
0,session_id,2022
1,Target,target
2,Original Data,"(376096, 16)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,8
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(376096, 25)"


In [23]:
pycaret_regression_models = compare_models(n_select=25, sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,3.6629,27.7698,5.2696,0.8908,0.1580,0.1102,14.5780
et,Extra Trees Regressor,4.0217,34.2808,5.8548,0.8652,0.1739,0.1199,19.4270
xgboost,Extreme Gradient Boosting,4.1937,32.2140,5.6756,0.8733,0.1752,0.1306,0.5190
dt,Decision Tree Regressor,4.7436,48.1181,6.9367,0.8108,0.2100,0.1397,1.6700
knn,K Neighbors Regressor,4.7455,43.7661,6.6154,0.8279,0.1933,0.1427,14.1120
lightgbm,Light Gradient Boosting Machine,5.1922,46.0946,6.7892,0.8188,0.2072,0.1639,119.2860
gbr,Gradient Boosting Regressor,7.1768,84.4137,9.1876,0.6681,0.2696,0.2295,28.4550
huber,Huber Regressor,9.5564,143.0902,11.9619,0.4374,0.3369,0.3005,6.5020
ridge,Ridge Regression,9.5802,142.6963,11.9454,0.4389,0.3354,0.3009,0.0600
lar,Least Angle Regression,9.5802,142.6963,11.9454,0.4389,0.3354,0.3009,0.0730


In [39]:
pycaret_regression_models2 = compare_models(n_select=25, sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,3.6629,27.7698,5.2696,0.8908,0.1580,0.1102,14.6160
et,Extra Trees Regressor,4.0217,34.2808,5.8548,0.8652,0.1739,0.1199,19.6030
xgboost,Extreme Gradient Boosting,4.1937,32.2140,5.6756,0.8733,0.1752,0.1306,0.4950
dt,Decision Tree Regressor,4.7436,48.1181,6.9367,0.8108,0.2100,0.1397,1.6450
knn,K Neighbors Regressor,4.7455,43.7661,6.6154,0.8279,0.1933,0.1427,14.8370
lightgbm,Light Gradient Boosting Machine,5.1922,46.0946,6.7892,0.8188,0.2072,0.1639,119.4570
gbr,Gradient Boosting Regressor,7.1768,84.4137,9.1876,0.6681,0.2696,0.2295,28.6730
huber,Huber Regressor,9.5564,143.0902,11.9619,0.4374,0.3369,0.3005,5.9750
ridge,Ridge Regression,9.5802,142.6963,11.9454,0.4389,0.3354,0.3009,0.0620
lar,Least Angle Regression,9.5802,142.6963,11.9454,0.4389,0.3354,0.3009,0.0720
